In [ ]:
!pip install flask-ngrok python-dotenv requests flask

In [ ]:
# Create necessary directories
!mkdir -p templates

In [ ]:
%%writefile templates/index.html
# Copy the contents of your index.html file here

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import requests
import os
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

FLUX_API_KEY = 'your_flux_api_key_here'

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate_image():
    prompt = request.form.get('prompt')
    
    headers = {
        'Authorization': f'Bearer {FLUX_API_KEY}',
        'Content-Type': 'application/json'
    }
    
    data = {
        'prompt': prompt,
        'samples': 1,
        'negative_prompt': 'blurry, low quality, distorted',
        'steps': 30,
        'cfg_scale': 7.5
    }
    
    try:
        response = requests.post(
            'https://api.flux.ai/v1/generate/text-to-image',
            headers=headers,
            json=data,
            timeout=30
        )
        
        if response.status_code == 200:
            image_data = response.json()['images'][0]
            return jsonify({'success': True, 'image': image_data})
        else:
            return jsonify({'success': False, 'error': f'API Error: {response.text}'})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

if __name__ == '__main__':
    app.run()

In [ ]:
# Run the Flask app
!python app.py